In [1]:
# input: cellsnp output with unwanted cols removed
# part 1: create defaultdict with d[barcode_index]((position, count)); output the dict
# part 2: reduce dict with duplicate vals (barcode multiplets)
# to contain only the keys with unique values; output the cleaned dict
# if barcode_indices have identical 1) and 2) then they are considered barcode multiplets and removed


from collections import defaultdict

def create_dict(in_file):
    # create dict of sets for each barcode_id
    d = defaultdict(set)
    
    with open(in_file, 'r') as f:
        # skip first three rows
        next(f)
        next(f)
        next(f)
        
        # format of in_file
        # 1 29195 1
        # 1 93378 2
        # position_id \t barcode_id \t count \n
        
        for line in f:
            line = line.strip('\n').split('\t')
            # add (position_id,count) to each barcode_id
            # using a set means no (position_id,count) duplicates will be added to the same barcode
            d[line[1]].add((line[0], int(line[2])))
    
    f.close()
    print(f'Number of cells before dup and zero removal: {len(d)}')
    
    return d


# remove duplicates based on 1) marker positions and 2) per marker position mapping counts

def remove_duplicates_and_zeros(d):
    # empty duplicates and retain unique
    count_dup = 0#
    for i,value in enumerate(d.values()):
        # search for (position_id,count) duplicates between barcodes
        # if counted more than one pair between barcodes
        if list(d.values()).count(value) > 1:
            # update counter
            count_dup += 1
            # remove the content of the current barcode and preserve the content of the other barcode(s)
            # this will remove the content of all barcodes with duplicate contents, until one remain
            d[list(d.keys())[i]] = ()
    
    # how many cells had multiple barcodes?
    print(f'Number of duplicate keys: {count_dup}')#
    
    
    # create tuple to contain barcodes with no content
    # either due to no content or removal in prev step
    zero_keys = ()
    for i,value in enumerate(d.values()):
        # if a key's value is empty, add key to the tuple
        # the only way to grab key from evaluating value is by enumerating the keys/values
        if not value:
            zero_keys += (list(d.keys())[i],)
            
    print(f'Number of zero keys after emptying duplicates: {len(zero_keys)}')#
    
    # rm zero_keys from output dict
    for key in zero_keys:
        d.pop(key)
    
    print(f'Number of cells after dup and zero removal: {len(d)}')#
    
    return d


In [4]:
# run on real dataset

#for X in ('A','B','C'):
for X in ('B'):
    
    in_file = '/Users/shell/MSc/LMU/Schneeberger/data/{}/cellSNP.tag.DP.mtx'.format(X)
    
    d = create_dict(in_file)
    avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
    # I think I could just use sum(d.values())/len(d.values())
    # oh, but this requires the dict value to be an int, whereas it is a tup now
    with open('/Users/shell/MSc/LMU/Schneeberger/out/{}_avgs0.csv'.format(X), 'w+') as fw:
        for pair in avgs:
            fw.write(f'{pair[0]},{pair[1]}\n')
    
    # remove
    d = remove_duplicates_and_zeros(d)
    avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
    with open('/Users/shell/MSc/LMU/Schneeberger/out/{}_avgs1.csv'.format(X), 'w+') as fw:
        for pair in avgs:
            fw.write(f'{pair[0]},{pair[1]}\n')
            
    print(X)


Number of cells before dup and zero removal: 387366


KeyboardInterrupt: 

In [ ]:
Number of cells before dup and zero removal: 273273
Number of duplicate keys: 20481
Number of zero keys after emptying duplicates: 20481
Number of cells after dup and zero removal: 252792
A

In [2]:
# run on test dataset: small

in_file = '/Users/shell/MSc/LMU/Schneeberger/cellSNP_small.mtx'
d = create_dict(in_file)
avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open('/Users/shell/MSc/LMU/Schneeberger/small_avgs0.csv', 'w+') as fw:
    fw.write('barcode_index,avg_count\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')

# remove
d = remove_duplicates_and_zeros(d)

avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open('/Users/shell/MSc/LMU/Schneeberger/small_avgs1.csv', 'w+') as fw:
    fw.write('barcode_index,avg_count\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')

Number of cells before dup and zero removal: 1013
Number of duplicate keys: 578
Number of zero keys after emptying duplicates: 578
Number of cells after dup and zero removal: 435


In [39]:
# run on test dataset: tiny

in_file = '/Users/shell/MSc/LMU/Schneeberger/tiny_cellsnp.mtx'
d = create_dict(in_file)
# for each key in d
# add tuple to list
# 1st element is int(key): barcode_index as int
# 2nd element is sum([tup[1] for tup in d[key]])/len(d[key]): avg read count per barcode
avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open('/Users/shell/MSc/LMU/Schneeberger/tiny_avgs0.csv', 'w+') as fw:
    fw.write('barcode_index,avg_count\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')

# remove
d = remove_duplicates_and_zeros(d)

avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open('/Users/shell/MSc/LMU/Schneeberger/tiny_avgs1.csv', 'w+') as fw:
    fw.write('barcode_index,avg_count\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')


Number of cells before dup and zero removal: 4
Number of duplicate keys: 1
Number of zero keys after emptying duplicates: 1
Number of cells after dup and zero removal: 3


In [29]:
d

defaultdict(set,
            {'29196': {('1', 1), ('2', 1)},
             '29198': {('1', 1), ('1', 2)},
             '15533': {('1', 1)}})

In [20]:
for pair in avgs:
    print(pair)

1.0
1.5
1.0


In [ ]:
#!/dss/dsshome1/08/ra65mav/.conda/envs/py3.11/bin/python
'''
This script generates avg depth with and without removal of barcode multiplets
# input: unmodified cellsnp output
# part 1: create defaultdict with d[barcode_index]((position, count)); output the dict
# part 2: reduce dict with duplicate vals (barcode multiplets)
# part 3: write avg depth per barcode_index, write to file
# to contain only the keys with unique values; output the cleaned dict
'''

import sys
from collections import defaultdict


# sys.argv[0] is the name of the python script
cellsnp_file_path = sys.argv[1]
output1_file_path = sys.argv[2]
output2_file_path = sys.argv[3]

# Example file paths
# cellsnp_file_path = '/dss/dsslegfs01/pn29fi/pn29fi-dss-0008/Yuying/cellsnp_lite_results/B_nigra_paternal_5733_B/cellSNP.tag.DP.mtx'
# output1_file_path = '/dss/dsslegfs01/pn29fi/pn29fi-dss-0008/Yuying/out/B_avgs.csv'
# output2_file_path = '/dss/dsslegfs01/pn29fi/pn29fi-dss-0008/Yuying/out/B_avgs_dup_rm.csv'

print(f"cellsnp_file_path = '{cellsnp_file_path}';")
print(f"output1_file_path (all cells) = '{output1_file_path}';")
print(f"output2_file_path (after barcode multiplet removal) = '{output2_file_path}'.")



def create_dict(in_file):
    # create dict of sets for each barcode_id
    d = defaultdict(set)
    
    with open(in_file, 'r') as f:
        # skip first three rows
        next(f)
        next(f)
        next(f)
        # format of in_file: position_id \t barcode_id \t count \n
        
        for line in f:
            line = line.strip('\n').split('\t')
            # add (position_id,count) to each barcode_id
            # using a set means no (position_id,count) duplicates will be added to the same barcode
            d[line[1]].add((line[0], int(line[2])))
    
    f.close()
    print(f'Number of cells before dup and zero removal: {len(d)}')
    
    return d


# remove duplicates based on 1) marker positions and 2) per marker position mapping counts
# if barcode_indices have identical 1) and 2) then they are considered barcode multiplets and removed

def remove_duplicates_and_zeros(d):
    # empty duplicates and retain unique
    count_dup = 0#
    for i,value in enumerate(d.values()):
        # search for (position_id,count) duplicates between barcodes
        # if counted more than one pair between barcodes
        if list(d.values()).count(value) > 1:
            # update counter
            count_dup += 1
            # remove the content of the current barcode and preserve the content of the other barcode(s)
            # this will remove the content of all barcodes with duplicate contents, until one remain
            d[list(d.keys())[i]] = ()
    
    # how many cells had multiple barcodes?
    print(f'Number of duplicate keys: {count_dup}')#
    
    # create tuple to contain barcodes with no content
    # either due to no content or removal in prev step
    zero_keys = ()
    for i,value in enumerate(d.values()):
        # if a key's value is empty, add key to the tuple
        # the only way to grab key from evaluating value is by enumerating the keys/values
        if not value:
            zero_keys += (list(d.keys())[i],)
            
    print(f'Number of zero keys after emptying duplicates: {len(zero_keys)}')#
    
    # rm zero_keys from output dict
    for key in zero_keys:
        d.pop(key)
    
    print(f'Number of cells after dup and zero removal: {len(d)}')#
    
    return d


# write output to file
d = create_dict(cellsnp_file_path)
avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open(output1_file_path, 'w+') as fw:
    fw.write('position_index,avg_depth\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')
print('done 1/2')

d = remove_duplicates_and_zeros(d)
avgs = [(int(key), sum([tup[1] for tup in d[key]])/len(d[key])) for key in d]
with open('/Users/shell/MSc/LMU/Schneeberger/out/{}_avgs1.csv', 'w+') as fw:
    fw.write('position_index,avg_depth\n')
    for pair in avgs:
        fw.write(f'{pair[0]},{round(pair[1],2)}\n')
print('done 2/2')